# Logreg train - Python 42

In [18]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import copy

In [20]:
data_raw1 = pd.read_csv("dataset_train.csv")
data_raw2 = pd.read_csv("dataset_test.csv")
data_raw = pd.concat([data_raw1, data_raw2])
data_raw_1 = data_raw.drop(['Index'], axis=1)
data_raw_1.head()

Hogwarts House First Name  Last Name    Birthday Best Hand  Arithmancy  \
0      Ravenclaw     Tamara        Hsu  2000-03-30      Left     58384.0   
1      Slytherin      Erich    Paredes  1999-10-14     Right     67239.0   
2      Ravenclaw   Stephany      Braun  1999-11-03      Left     23702.0   
3     Gryffindor      Vesta  Mcmichael  2000-08-19      Left     32667.0   
4     Gryffindor     Gaston      Gibbs  1998-09-27      Left     60158.0   

    Astronomy  Herbology  Defense Against the Dark Arts  Divination  \
0 -487.886086   5.727180                       4.878861       4.722   
1 -552.060507  -5.987446                       5.520605      -5.612   
2 -366.076117   7.725017                       3.660761       6.140   
3  697.742809  -6.497214                      -6.977428       4.026   
4  436.775204  -7.820623                            NaN       2.236   

   Muggle Studies  Ancient Runes  History of Magic  Transfiguration   Potions  \
0      272.035831     532.484226          5.231058      1039.788281  3.790369   
1     -487.340557     367.760303          4.107170      1058.944592  7.248742   
2      664.893521     602.585284          3.555579      1088.088348  8.728531   
3     -537.001128     523.982133         -4.809637       920.391449  0.821911   
4     -444.262537     599.324514         -3.444377       937.434724  4.311066   

   Care of Magical Creatures     Charms  Flying  
0                   0.715939 -232.79405  -26.89  
1                   0.091674 -252.18425 -113.45  
2                  -0.515327 -227.34265   30.42  
3                  -0.014040 -256.84675  200.64  
4                  -0.264070 -256.38730  157.98

In [3]:
results = data_raw_1.describe()
results

Arithmancy    Astronomy    Herbology  Defense Against the Dark Arts  \
count    1953.000000  1955.000000  1956.000000                    1961.000000   
mean    49724.612903    41.451669     1.189644                      -0.417844   
std     16444.807977   518.651589     5.187046                       5.193183   
min    -24370.000000  -966.740546   -10.295663                     -10.162119   
25%     38819.000000  -488.265275    -4.300239                      -5.243928   
50%     49114.000000   272.071636     3.506840                      -2.707410   
75%     60698.000000   521.974961     5.416000                       4.881403   
max    104956.000000  1016.211940    11.612895                       9.667405   

        Divination  Muggle Studies  Ancient Runes  History of Magic  \
count  1955.000000     1955.000000    1957.000000       1946.000000   
mean      3.205737     -223.708112     495.785943          2.936453   
std       4.103768      488.499430     105.348963          4.403510   
min      -8.727000    -1086.496835     283.869609         -8.858993   
25%       3.182500     -577.927473     398.497892          2.219926   
50%       4.634000     -415.425616     467.730624          4.355191   
75%       5.655000      252.532155     596.599814          5.780673   
max      10.032000     1092.388611     745.396220         11.889713   

       Transfiguration      Potions  Care of Magical Creatures       Charms  \
count      1955.000000  1960.000000                1952.000000  2000.000000   
mean       1030.253905     5.915848                  -0.038082  -243.335749   
std          44.360287     3.161791                   0.981402     8.770542   
min         906.627320    -4.697484                  -3.313676  -261.048920   
25%        1026.510177     3.554956                  -0.660419  -250.586450   
50%        1045.785090     5.857253                  -0.025599  -244.831995   
75%        1058.649546     8.236317                   0.621809  -232.598710   
max        1099.966073    13.536762                   3.205525  -225.428140   

           Flying  
count  2000.00000  
mean     22.23990  
std      97.12546  
min    -181.47000  
25%     -40.71250  
50%      -2.51500  
75%      49.45750  
max     282.43000

Code for the logistic regression

In [63]:
class logreg:
    
    def __init__(self, lr=0.01, iterations=100000, fit_intercept=True):
        self.lr = lr
        self.iterations = iterations
        self.fit_intercept = fit_intercept
        
    def __add_intercept(self,X):
        cols = X.columns
        X['One'] = 1
        return X.filter(['One'] + cols)
        
    def __sigmoid(z):
        return 1/(1+np.exp(-z))
        
    def __loss(y,h):
        return (-y*np.log(h) - (1-y)*np.log(1-h))
        
    def __fit(self, X, y):
        if self.fit_intercept:
            X = self.add_intercept(X)
    
        #We initialize the theta
        self.theta = np.zeros(X.shape[1])
        
        for i in np.arange(0, self.iterations):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = 1/y.size * np.dot(X.T, (h-y))
            self.theta -= lr*gradient
        
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold):
        return self.predict_prob(X) >= threshold

As we want to have a One-versus-all approach, we divide the "Hogwarts House" category into dummy variables - each representing one house. 

In [31]:
houses = pd.get_dummies(data_raw_1['Hogwarts House'])
new_data = pd.concat([data_raw_1, houses], axis = 1)
new_data.head()

Hogwarts House First Name  Last Name    Birthday Best Hand  Arithmancy  \
0      Ravenclaw     Tamara        Hsu  2000-03-30      Left     58384.0   
1      Slytherin      Erich    Paredes  1999-10-14     Right     67239.0   
2      Ravenclaw   Stephany      Braun  1999-11-03      Left     23702.0   
3     Gryffindor      Vesta  Mcmichael  2000-08-19      Left     32667.0   
4     Gryffindor     Gaston      Gibbs  1998-09-27      Left     60158.0   

    Astronomy  Herbology  Defense Against the Dark Arts  Divination  \
0 -487.886086   5.727180                       4.878861       4.722   
1 -552.060507  -5.987446                       5.520605      -5.612   
2 -366.076117   7.725017                       3.660761       6.140   
3  697.742809  -6.497214                      -6.977428       4.026   
4  436.775204  -7.820623                            NaN       2.236   

     ...      History of Magic  Transfiguration   Potions  \
0    ...              5.231058      1039.788281  3.790369   
1    ...              4.107170      1058.944592  7.248742   
2    ...              3.555579      1088.088348  8.728531   
3    ...             -4.809637       920.391449  0.821911   
4    ...             -3.444377       937.434724  4.311066   

   Care of Magical Creatures     Charms  Flying  Gryffindor  Hufflepuff  \
0                   0.715939 -232.79405  -26.89           0           0   
1                   0.091674 -252.18425 -113.45           0           0   
2                  -0.515327 -227.34265   30.42           0           0   
3                  -0.014040 -256.84675  200.64           1           0   
4                  -0.264070 -256.38730  157.98           1           0   

   Ravenclaw  Slytherin  
0          1          0  
1          0          1  
2          1          0  
3          0          0  
4          0          0  

[5 rows x 22 columns]

In [68]:
model_gryffindor = logreg()

In [65]:
new_data['Hogwarts House'].unique()

array(['Ravenclaw', 'Slytherin', 'Gryffindor', 'Hufflepuff', nan],
      dtype=object)

In [66]:
X = new_data.drop(['Hogwarts House', 'First Name', 'Last Name', 'Birthday', 'Best Hand', 'Gryffindor', 'Ravenclaw', 'Hufflepuff', 'Slytherin'], axis = 1)
y = new_data.Gryffindor

In [69]:
model_gryffindor.fit(X,y)

AttributeError: 'logreg' object has no attribute 'fit'

In [54]:
model = LogisticRegression(lr=0.1, num_iter=300000)
X = data_raw1.drop(['Hogwarts House', 'First Name', 'Last Name', 'Birthday', 'Best Hand'], axis = 1)
y = data_raw1['Hogwarts House']
%time model.fit(X, y)  
preds = model.predict(X, 0.3)
# accuracy
(preds == y).mean()
1.0

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

/Users/louisdussart/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in greater_equal


1.0

In [48]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
            
            if(self.verbose == True and i % 10000 == 0):
                z = np.dot(X, self.theta)
                h = self.__sigmoid(z)
                print(f'loss: {self.__loss(h, y)} \t')
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold):
        return self.predict_prob(X) >= threshold

In [10]:
model = LogisticRegression(lr=0.1, num_iter=300000)
X = data_raw1.drop(['Hogwarts House', 'First Name', 'Last Name', 'Birthday', 'Best Hand'], axis = 1)
y = data_raw1['Hogwarts House']
%time model.fit(X, y)  
preds = model.predict(X, 0.3)
# accuracy
(preds == y).mean()
1.0

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

/Users/louisdussart/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in greater_equal


1.0